First, import needed modules

In [23]:
import numpy as np
import pandas as pd
import os
from functools import reduce

Initalize variables

In [24]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [25]:
import chardet #check encoding of files
with open(f'{path}/data/controls/Altersgruppen_2012.csv', 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [26]:
employed = pd.read_csv(f'{path}/data/controls/Beschaeftigte_2012.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.'])
unemployed = pd.read_csv(f'{path}/data/controls/Arbeitslose_2012.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.'])
age_groups  = pd.read_csv(f'{path}/data/controls/Altersgruppen_2012.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.'])
age_avg  = pd.read_csv(f'{path}/data/controls/Durchschnittsalter_2012.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.'])
income  = pd.read_csv(f'{path}/data/controls/Einkuenfte_2010.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.'])
area  = pd.read_csv(f'{path}/data/controls/Flaeche_2011.csv', encoding = 'ISO-8859-1', sep=';', na_values=[' - ', 'x', '.'])
gender_nationality  = pd.read_csv(f'{path}/data/controls/Geschlecht_Nationalitaet_2011.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.'])
religion  = pd.read_csv(f'{path}/data/controls/Religion_2011.csv', encoding = 'ISO-8859-1', sep=';', na_values=['-', 'x', '.', '0'])

Inspect datasets

In [27]:
age_groups.head(10)

,AGS,age 0-17,age 18-24,age 25-44,age 45-64,age 65+
0,1,"16,7","7,7","23,5","29,9","22,1"
1,1001,"15,2","11,3","26,3","26,6","20,6"
2,1002,"14,3","11,6","30,2","25,6","18,3"
3,1003,15,"8,4","25,4","28,1","23,1"
4,1004,"16,6","8,7","23,6","28,9","22,3"
5,1051,"17,3","7,5","21,7","30,3","23,3"
6,1051001,20,"7,6","21,6","27,8","23,1"
7,1051002,"20,1","4,7","26,2",35,14
8,1051003,"16,7","9,8","21,1",35,"17,4"
9,1051004,"15,7","7,7","21,6","29,5","25,5"


In [28]:
age_groups.dtypes

AGS           int64
age 0-17     object
age 18-24    object
age 25-44    object
age 45-64    object
age 65+      object
dtype: object

Pivot religion table

In [29]:
religion_pivot = pd.pivot_table(religion, index=['AGS'], columns=['Religion'], values='Bevölkerung', aggfunc=np.sum, fill_value=0)
religion_pivot.rename(columns = {'Evangelische Kirche':'Evangelisch', 'Römisch-katholische Kirche': 'Katholisch', 'Sonstige, keine, ohne Angabe': 'Religion Sonstige/Keine'}, inplace = True)
religion_pivot = religion_pivot[['Evangelisch', 'Katholisch', 'Religion Sonstige/Keine']]
religion_pivot.head()

Religion,Evangelisch,Katholisch,Religion Sonstige/Keine
AGS,,,
1,1442840,167565,1189714
2,507877,167583,1031236
3,3777068,1353460,2647464
4,261477,70999,318387
5,4745176,7164962,5628114


Merge datasets

In [36]:
dfs = [area, gender_nationality, age_avg, age_groups, employed, unemployed, income, religion_pivot]
controls = reduce(lambda  left, right: pd.merge(left, right, on=['AGS'], how='outer'), dfs)
controls.head()

,AGS,Area,Bevölkerung,männlich,weiblich,Ausländer/-innen,avg. age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,Beschäftigte,Arbeitslose,Einkünfte je Steuerpflichtigen in Tsd.,Evangelisch,Katholisch,Religion Sonstige/Keine
0,1,"15799,56",2800119.0,1360484.0,1439635.0,117059.0,"44,5","16,7","7,7","23,5","29,9","22,1",973162.0,100048.0,"32,2",1442840,167565,1189714
1,1001,"56,74",82258.0,40534.0,41724.0,5129.0,43,"15,2","11,3","26,3","26,6","20,6",27016.0,5127.0,27,40867,5220,36171
2,1002,"118,65",235782.0,113505.0,122277.0,16571.0,"41,8","14,3","11,6","30,2","25,6","18,3",80005.0,12732.0,"27,5",97355,17368,121059
3,1003,"214,21",210305.0,99832.0,110473.0,13157.0,"44,9",15,"8,4","25,4","28,1","23,1",69777.0,11009.0,"27,6",101500,17793,91012
4,1004,"71,63",77249.0,37710.0,39539.0,4661.0,"44,4","16,6","8,7","23,6","28,9","22,3",25751.0,4376.0,"26,2",38356,5019,33874


Fix the AGS

In [31]:
controls['AGS'] = controls['AGS'].astype(str)
controls['AGS'] = np.where(controls['AGS'].str.len() <= 2, controls['AGS'] + '0'*6, controls['AGS']) # add 6*0 after AGS for states with 1 or 2 digit AGS
controls['AGS'] = np.where(controls['AGS'].str.len() < 6, controls['AGS'] + '0'*3, controls['AGS']) # add trailing 000 for kreisfreie Städte (AGS has 4-5 digits before, 7-8 after)
controls['AGS'] = np.where(controls['AGS'].str.len() < 8, '0' + controls['AGS'], controls['AGS']) # add leading 0 for state ids < 10 (AGS has 7 digits)
controls.head()
# Note: there are some obs with 3 digit AGS (141-147, 151-153) which cannot be identified. Likely these are statistical regions in Saxony/Saxony-Anhalt

,AGS,Area,Bevölkerung,männlich,weiblich,Ausländer/-innen,avg. age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,Beschäftigte,Arbeitslose,Einkünfte je Steuerpflichtigen in Tsd.,Evangelisch,Katholisch,Religion Sonstige/Keine
0,01000000,"15799,56",2800119.0,1360484.0,1439635.0,117059.0,"44,5","16,7","7,7","23,5","29,9","22,1",973162.0,100048.0,"32,2",1442840,167565,1189714
1,01001000,"56,74",82258.0,40534.0,41724.0,5129.0,43,"15,2","11,3","26,3","26,6","20,6",27016.0,5127.0,27,40867,5220,36171
2,01002000,"118,65",235782.0,113505.0,122277.0,16571.0,"41,8","14,3","11,6","30,2","25,6","18,3",80005.0,12732.0,"27,5",97355,17368,121059
3,01003000,"214,21",210305.0,99832.0,110473.0,13157.0,"44,9",15,"8,4","25,4","28,1","23,1",69777.0,11009.0,"27,6",101500,17793,91012
4,01004000,"71,63",77249.0,37710.0,39539.0,4661.0,"44,4","16,6","8,7","23,6","28,9","22,3",25751.0,4376.0,"26,2",38356,5019,33874


Compute statistics (first convert dtypes of some columns)

In [32]:
controls.dtypes

AGS                                        object
Area                                       object
Bevölkerung                               float64
männlich                                  float64
weiblich                                  float64
Ausländer/-innen                          float64
avg. age                                   object
age 0-17                                   object
age 18-24                                  object
age 25-44                                  object
age 45-64                                  object
age 65+                                    object
Beschäftigte                              float64
Arbeitslose                               float64
Einkünfte je Steuerpflichtigen in Tsd.     object
Evangelisch                                 int64
Katholisch                                  int64
Religion Sonstige/Keine                     int64
dtype: object

In [33]:
controls['Area'] = controls['Area'].str.replace(',','.').astype(float)
controls['avg_age'] = controls['avg. age'].str.replace(',','.').astype(float)
controls['age 0-17'] = controls['age 0-17'].str.replace(',','.').astype(float)
controls['age 18-24'] = controls['age 18-24'].str.replace(',','.').astype(float)
controls['age 25-44'] = controls['age 25-44'].str.replace(',','.').astype(float)
controls['age 45-64'] = controls['age 45-64'].str.replace(',','.').astype(float)
controls['age 65+'] = controls['age 65+'].str.replace(',','.').astype(float)
controls['avg_income'] = controls['Einkünfte je Steuerpflichtigen in Tsd.'].str.replace(',','.').astype(float)*1000
controls.dtypes

AGS                                        object
Area                                      float64
Bevölkerung                               float64
männlich                                  float64
weiblich                                  float64
Ausländer/-innen                          float64
avg. age                                   object
age 0-17                                  float64
age 18-24                                 float64
age 25-44                                 float64
age 45-64                                 float64
age 65+                                   float64
Beschäftigte                              float64
Arbeitslose                               float64
Einkünfte je Steuerpflichtigen in Tsd.     object
Evangelisch                                 int64
Katholisch                                  int64
Religion Sonstige/Keine                     int64
avg_age                                   float64
avg_income                                float64


In [34]:
controls['pop_density'] = controls['Bevölkerung'] / controls['Area']
controls['male'] = controls['männlich'] / controls['Bevölkerung']
controls['female'] = controls['weiblich'] / controls['Bevölkerung']
controls['foreign'] = controls['Ausländer/-innen'] / controls['Bevölkerung']
controls['employed'] = controls['Beschäftigte'] / controls['Bevölkerung']
controls['unemployed'] = controls['Arbeitslose'] / controls['Bevölkerung']
controls['protestant'] = controls['Evangelisch'] / controls['Bevölkerung']
controls['catholic'] = controls['Katholisch'] / controls['Bevölkerung']
controls['religion_other'] = controls['Religion Sonstige/Keine'] / controls['Bevölkerung']
controls = controls[['AGS', 'pop_density', 'male', 'female', 'foreign', 'employed', 'unemployed', 'avg_income', 'avg_age', 'age 0-17', 'age 18-24', 'age 25-44', 'age 45-64', 'age 65+', 'protestant', 'catholic', 'religion_other']]
controls.head()

,AGS,pop_density,male,female,foreign,employed,unemployed,avg_income,avg_age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,protestant,catholic,religion_other
0,01000000,177.227657,0.485866,0.514134,0.041805,0.347543,0.035730,32200.0,44.5,16.7,7.7,23.5,29.9,22.1,0.515278,0.059842,0.424880
1,01001000,1449.735636,0.492767,0.507233,0.062353,0.328430,0.062328,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
2,01002000,1987.206068,0.481398,0.518602,0.070281,0.339318,0.053999,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436
3,01003000,981.770225,0.474701,0.525299,0.062562,0.331790,0.052348,27600.0,44.9,15.0,8.4,25.4,28.1,23.1,0.482632,0.084606,0.432762
4,01004000,1078.444786,0.488162,0.511838,0.060337,0.333351,0.056648,26200.0,44.4,16.6,8.7,23.6,28.9,22.3,0.496524,0.064972,0.438504


Export dataset

In [35]:
controls = controls.set_index('AGS')
controls.to_csv(f'{path}/data/controls.csv', encoding = 'utf-8-sig')
controls.head()

,pop_density,male,female,foreign,employed,unemployed,avg_income,avg_age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,protestant,catholic,religion_other
AGS,,,,,,,,,,,,,,,,
01000000,177.227657,0.485866,0.514134,0.041805,0.347543,0.035730,32200.0,44.5,16.7,7.7,23.5,29.9,22.1,0.515278,0.059842,0.424880
01001000,1449.735636,0.492767,0.507233,0.062353,0.328430,0.062328,27000.0,43.0,15.2,11.3,26.3,26.6,20.6,0.496815,0.063459,0.439726
01002000,1987.206068,0.481398,0.518602,0.070281,0.339318,0.053999,27500.0,41.8,14.3,11.6,30.2,25.6,18.3,0.412903,0.073661,0.513436
01003000,981.770225,0.474701,0.525299,0.062562,0.331790,0.052348,27600.0,44.9,15.0,8.4,25.4,28.1,23.1,0.482632,0.084606,0.432762
01004000,1078.444786,0.488162,0.511838,0.060337,0.333351,0.056648,26200.0,44.4,16.6,8.7,23.6,28.9,22.3,0.496524,0.064972,0.438504
